# 程序编译

``BrainState`` 通过 ``brainstate.compile`` 实现了跨硬件的编译部署，可以将Python代码转换为程序中间表示（IR），并在不同硬件上进行编译和优化。

``brainstate``中提供的编译支持主要集成在``brainstate.compile``中。这些编译APIs囊括了一系列语法功能，包括：

- 即时编译： 支持 JIT 即时编译，提高计算效率和性能。
- 条件语句： 支持 if-else 逻辑，方便用户根据不同条件执行不同的计算流程。
- 循环语句： 支持 for，while 循环，方便用户重复执行相同的计算操作。

JAX框架本身提供了非常实用的转换方法（[Transformations](https://jax.readthedocs.io/en/latest/key-concepts.html#transformations)）来优化程序的性能。一般来说，JAX 转换在 ``jax.Arrays`` 的 ``pytrees`` 上运行，并遵循值语义。BrainPy 中的大部分计算都依赖于 JAX。JAX 为 Python 程序提供了出色的变换，包括微分、向量化、并行化和即时编译。然而，JAX 的转换是基于函数的，而不是基于状态的。这意味着，JAX 无法对``State``进行优化，因为``State``是在运行时动态生成的。为了解决这个问题，BrainPy 提供了 ``brainstate.compile``，它可以将``State``编译为计算图，从而实现对``State``的优化，这些编译集扩展了 JAX 转换，允许 ``State`` 对象传入和传出转换。

brainstate编译的一大特色是，它只对``State``感知：在程序运行过程中，只要遇到一个``State``实例，就会将其编译进计算图，然后在不同硬件上运行。这种编译方式使得用户能够任意定义复杂的程序，而编译器会根据程序的实际运行分支进行针对性的优化，以此极大提高计算效率。同时，只对``State``感知的编译模式还使得用户能够更灵活地表达程序逻辑，而不用在意``PyGraph``、``PyTree``等概念的限制，从而彻底释放编程的灵活性。下面我们将介绍``brainstate.compile``的具体用法。


In [9]:
import jax.numpy as jnp
import jax
import brainstate

## 即时编译
``brainstate.compile`` 支持 JIT 即时编译，提高计算效率和性能。用户可以使用``brainstate.compile.jit``对函数进行即时编译，以提高计算效率。下面是一个例子：

In [5]:
a = brainstate.State(brainstate.random.randn(10))

@brainstate.compile.jit
def fun1(inp):
    a.value += inp

    b = brainstate.State(brainstate.random.randn(1))

    def inner_fun(x):
        b.value += x

    brainstate.compile.for_loop(inner_fun, brainstate.random.randn(100))

    return a.value + b.value

x = brainstate.random.randn(10)
print(fun1(x))
key = fun1.stateful_fun.get_arg_cache_key(x)
fun1.stateful_fun.get_states(key)

[1.9970131 1.8363075 1.2407477 3.2610366 2.4621444 0.8094802 3.0505526
 3.3499    0.5930283 1.8049405]


(State(
   value=Array([-0.47143596, -0.63214153, -1.2277014 ,  0.79258746, -0.00630462,
          -1.6589689 ,  0.5821035 ,  0.88145095, -1.8754208 , -0.6635086 ],      dtype=float32)
 ),
 RandomState([1811064066  170626415]))

在上述代码中，我们定义了一个函数``fun1``，它接受一个输入``inp``，并在函数内部对``State``变量进行操作。在函数内部，我们定义了一个内部函数``inner_fun``，并在内部函数中对另一个``State``变量进行操作。在函数的最后，我们返回了两个``State``变量的和。在函数的最后，我们调用了``fun1``函数，并传入了一个随机的输入``x``。在函数调用之后，我们通过``fun1.stateful_fun.get_arg_cache_key``获取了函数调用的缓存键，并通过``fun1.stateful_fun.get_states``获取了函数调用的所有``State``变量。可以发现函数内部的局部状态变量`b`并没有被缓存，在构建计算图时会被优化掉。因此，只有定义在外部的`a`和随机数`RandomState`会被缓存，这样可以减少内存占用，提高计算效率。

需要特别注意的是，``brainstat.compile.jit`` 是会产生缓存的，因此被调用多次时，只有第一次会产生缓存，后续调用会直接使用缓存。如果需要重新编译，可以使用``fun1.stateful_fun.clear_cache()``清除缓存。

## 条件语句
``brainstate.compile`` 支持条件语句，用户可以根据不同的条件执行不同的计算流程。这里我们提供了``brainstate.compile.cond``，``brainstate.compile.switch``和``brainstate.compile.if_else``三种条件语句的编译函数。我们依次来看一下这三种条件语句的用法。

首先是``brainstate.compile.cond``，它的用法如下：

In [8]:
st1 = brainstate.State(brainstate.random.rand(10))
st2 = brainstate.State(brainstate.random.rand(2))
st3 = brainstate.State(brainstate.random.rand(5))
st4 = brainstate.State(brainstate.random.rand(2, 10))

def true_fun(x):
    st1.value = st2.value @ st4.value + x

def false_fun(x):
    st3.value = (st3.value + 1.) * x

brainstate.compile.cond(True, true_fun, false_fun, 2.)

如果参数类型正确，``cond()`` 的语义与 Python 的实现相当，其中 ``pred`` 必须是标量类型。

其次是``brainstate.compile.switch``，它的用法如下：

In [12]:
branches = [jax.lax.add, jax.lax.mul]

def cfun(x):
    return brainstate.compile.switch(x, branches, x, x)

print(cfun(2), cfun(-1))

4 -2


在上面的代码中，我们定义了一个函数``cfun``，它接受一个输入``x``，并根据``x``的值选择不同的分支函数。在函数的最后，我们返回了两个分支函数的计算结果。在函数调用之后，我们分别传入了``2``和``-1``两个输入，可以发现函数会根据输入的值选择不同的分支函数进行计算。

最后是``brainstate.compile.if_else``，它的用法如下：

In [13]:
def f(a):
    return brainstate.compile.ifelse(conditions=[a < 0,
                                                 a >= 0 and a < 2,
                                                 a >= 2 and a < 5,
                                                 a >= 5 and a < 10,
                                                 a >= 10],
                                     branches=[lambda: 1,
                                        lambda: 2,
                                        lambda: 3,
                                        lambda: 4,
                                        lambda: 5])

assert f(3) == 3
assert f(1) == 2
assert f(-1) == 1

在上面的代码中，我们定义了一个函数``f``，它接受一个输入``a``，并根据``a``的值选择不同的分支函数。在函数的最后，我们返回了选择的分支函数的计算结果。在函数调用之后，我们分别传入了``3``、``1``和``-1``三个输入，可以发现函数会根据输入的值选择不同的分支函数进行计算。

特别需要注意的是，条件语句并没有缓存的功能，每一次重新运行该函数都会需要重新编译一次。因此，如果需要多次调用，建议使用``brainstate.compile.jit``对函数进行即时编译。

## 循环语句
``brainstate.compile`` 支持for循环，用户可以重复执行相同的计算操作。这里我们提供了``brainstate.compile.for_loop``和``brainstate.compile.while_loop``两种循环语句的编译函数。我们依次来看一下这两种循环语句的用法。

首先是``brainstate.compile.for_loop``，它的用法如下：

In [14]:
a = brainstate.ShortTermState(0.)
b = brainstate.ShortTermState(0.)

def f(i):
    a.value += (1 + b.value)
    return a.value

n_iter = 10
ops = jnp.arange(n_iter)
r = brainstate.compile.for_loop(f, ops)

print(a)
print(b)

ShortTermState(
  value=Array(10., dtype=float32, weak_type=True)
)
ShortTermState(
  value=0.0
)


在上面的代码中，我们定义了一个函数``f``，它接受一个输入``i``，并在函数内部对``State``变量进行操作。在函数的最后，我们返回了``State``变量``a``的值。在函数调用之后，我们传入了一个``jnp.arange(n_iter)``的数组，这样函数会被循环调用``n_iter``次。在函数调用之后，我们打印了``State``变量``a``和``b``的值，可以发现``State``变量``a``的值是``a.value + n_iter * (1 + b.value)``。

``brainstate.compile.while_loop``的用法如下：

In [15]:
a = brainstate.State(1.)
b = brainstate.State(20.)

def cond(x):
    return a.value < b.value

def body(x):
    a.value += x
    return x

r = brainstate.compile.while_loop(cond, body, 1.)

print(a.value, b.value, r)

20.0 20.0 1.0


除了这两种循环语句，另一种较为常用的循环语句为``brainstate.compile.scan``，它的Python版本可以理解为：
```python
def scan(f, init, xs, length=None):
  if xs is None:
    xs = [None] * length
  carry = init
  ys = []
  for x in xs:
    carry, y = f(carry, x)
    ys.append(y)
  return carry, np.stack(ys)
```

与 Python 版本不同的是，`xs` 和 `ys` 都可以是任意的 `pytree` 值，因此可以同时扫描多个数组，并产生多个输出数组。`None` 实际上是一个特例，因为它代表一个空的 `pytree`。

另外，与 Python 版本不同的是，`scan()` 是一个 JAX 基元，可以降低到单个 `WhileOp`。这使得它在减少 JIT 编译函数的编译时间方面非常有用，因为 `jit()` 函数中的原生 Python 循环结构会被展开，从而导致大量的 XLA 计算。

最后还需要提醒的是，循环语句并没有缓存的功能，每一次重新运行该函数都会需要重新编译一次。因此，如果需要多次调用，建议使用``brainstate.compile.jit``对函数进行即时编译。

## 程序编译的嵌套使用
``BrainState``编译的另一个特色是，它能嵌套地调用无论是``JAX``提供的函数式的编译函数还是``BrainState``内置的``State``感知的编译函数。中间步骤生成或利用的``State``变量将只会是局部变量，在整个程序中将被优化掉。这种特性使得程序内存占用更小，运行速度更快。下面是一个例子：

In [2]:
b = brainstate.State(0.)


def add(i):
    c = brainstate.State(0.)

    def cond(j):
        return j <= i

    def body(j):
        c.value += 1.
        return j + 1

    brainstate.compile.while_loop(cond, body, 0.)

    b.value += c.value


brainstate.compile.for_loop(add, jnp.arange(10))

print(b.value)

55.0


在上面的程序中，我们定义了一个函数``add``，它会在循环中调用``while_loop``。在``while_loop``中，我们定义了一个局部变量``c``，并在循环中对其进行累加。在整个程序中，``c``只是一个局部变量，在编译过程中不会被缓存。这种嵌套调用的方式使得程序更加灵活，同时也能够保证程序的性能。值得注意的是，JAX所提供的转换也可以与``BrainState``的编译函数嵌套使用，这样可以更好地发挥JAX的优化能力。由于``BrainState``的编译函数需要对``State``进行感知，这些编译函数本身存在一定的编译开销（虽然很小），比起JAX的转换函数可能会慢一些，因此在实际使用中需要根据实际情况选择合适的编译函数。